In [1]:
import pandas as pd
import requests

In [41]:
df = pd.read_csv('./streetview/data/google_street_view_metadata/wayne2.csv')
df.head()

,id,segment_id,osm_id,osm_name,osm_type,start_lat,start_long,end_lat,end_long,gsv_metadata,copyright,date,location,pano_id,status,url_img0,url_img90,url_img180,url_img270
0,0,0,4413575,8 Mile Road,secondary,42.441909,-83.322163,42.441834,-83.321210,"{u'date': u'2016-08', u'status': u'OK', u'loca...",© 2017 Google,2016-08,"{u'lat': 42.4418379479, u'lng': -83.322204143}",qeY-1lyOWTTPHjQ3b3z_lA,OK,wayne2/img0_0000.jpg,wayne2/img90_0000.jpg,wayne2/img180_0000.jpg,wayne2/img270_0000.jpg
1,1,1,4688173,Eureka Road,secondary,42.192723,-83.348046,42.191222,-83.356192,"{u'date': u'2011-07', u'status': u'OK', u'loca...",© 2017 Google,2011-07,"{u'lat': 42.192674, u'lng': -83.3480385}",qXcYeFH-OPhZ4Ie1QYvH_g,OK,wayne2/img0_0001.jpg,wayne2/img90_0001.jpg,wayne2/img180_0001.jpg,wayne2/img270_0001.jpg
2,2,4,4767192,Dickerson Street,tertiary,42.369270,-82.952822,42.370532,-82.953648,"{u'date': u'2013-09', u'status': u'OK', u'loca...",© 2017 Google,2013-09,"{u'lat': 42.3692295762, u'lng': -82.9527694885}",ldcGQpqxWBjDd-G3XDeiaw,OK,wayne2/img0_0002.jpg,wayne2/img90_0002.jpg,wayne2/img180_0002.jpg,wayne2/img270_0002.jpg
3,3,5,4767193,Dickerson Street,tertiary,42.369270,-82.952822,42.366990,-82.950280,"{u'date': u'2013-09', u'status': u'OK', u'loca...",© 2017 Google,2013-09,"{u'lat': 42.3692295762, u'lng': -82.9527694885}",ldcGQpqxWBjDd-G3XDeiaw,OK,wayne2/img0_0003.jpg,wayne2/img90_0003.jpg,wayne2/img180_0003.jpg,wayne2/img270_0003.jpg
4,4,7,4798440,Ecorse Road,tertiary,42.249202,-83.436907,42.248952,-83.430874,"{u'date': u'2016-08', u'status': u'OK', u'loca...",© 2017 Google,2016-08,"{u'lat': 42.2492107254, u'lng': -83.4369273937}",c6eRt3Vso7pplQSwyJRToQ,OK,wayne2/img0_0004.jpg,wayne2/img90_0004.jpg,wayne2/img180_0004.jpg,wayne2/img270_0004.jpg


In [27]:
### FIXME: Get account ID and API key from https://askgeo.com/account
ACCOUNT_ID = 'ACCOUNT ID FROM ASKGEO'
API_KEY = 'ACCOUNT ID FROM ASKGEO '
BATCH_SIZE = 100

def get_askgeo(points, databases=['UsTract2010', 'UsZcta2010']):
    params = {'points': ';'.join(['%s,%s' % (r[0], r[1]) for r in points]),
                    'databases': ','.join(databases)
                   }
    r = requests.get('http://api.askgeo.com/v1/{0!s}/{1!s}/query.json'.format(ACCOUNT_ID, API_KEY), params=params)
    if r.status_code == 200:
        json = r.json()
        if json['message'] == 'ok':
            results = []
            for r in json['data']:
                data = {}
                for d in r:
                    for f in r[d]:
                        data[d + '_' + f] = r[d][f]
                results.append(data)
            return results

    return [{}]*len(points)

In [32]:
i =  0
data = []
while i < len(df):
    points = []
    for idx, r in df.iloc[i:i + BATCH_SIZE].iterrows():
        loc =  eval(r.location)
        points.append((loc['lat'], loc['lng']))
    i += BATCH_SIZE
    data.extend(get_askgeo(points))
    #break

In [40]:
df2 = pd.DataFrame(data)
df2.head()

,UsTract2010_AcsAgeFemaleMedianAge,UsTract2010_AcsAgeMaleMedianAge,UsTract2010_AcsAgeMedianAge,UsTract2010_AcsAgeUnder18,UsTract2010_AcsComponent,UsTract2010_AcsCountyCode,UsTract2010_AcsEducationFemale,UsTract2010_AcsEducationFemaleAssociatesDegree,UsTract2010_AcsEducationFemaleBachelorsDegree,UsTract2010_AcsEducationFemaleDoctorateDegree,...,UsZcta2010_FeatureClassCode,UsZcta2010_FunctionalStatus,UsZcta2010_GeoId,UsZcta2010_IsInside,UsZcta2010_LandAreaSqM,UsZcta2010_LatitudeDeg,UsZcta2010_LongitudeDeg,UsZcta2010_MinDistanceKm,UsZcta2010_WaterAreaSqM,UsZcta2010_ZctaCode
0,33.7,39.9,36.4,906,0,163,1263,77,186,10,...,G6350,S,48152,true,31368242,42.424935,-83.374264,0,33600,48152
1,32.3,33.8,32.8,2166,0,163,2024,203,97,0,...,G6350,S,48174,true,106680136,42.234298,-83.394324,0,588742,48174
2,41.8,38.4,40.0,919,0,163,1195,72,174,31,...,G6350,S,48215,true,10395926,42.375051,-82.954343,0,0,48215
3,41.8,38.4,40.0,919,0,163,1195,72,174,31,...,G6350,S,48215,true,10395926,42.375051,-82.954343,0,0,48215
4,36.3,31.2,34.3,2154,0,163,2259,183,369,0,...,G6350,S,48184,true,18208616,42.274479,-83.395623,0,3148,48184


In [39]:
df3 = pd.concat([df, df2], axis=1)
df3.head()

,id,segment_id,osm_id,osm_name,osm_type,start_lat,start_long,end_lat,end_long,gsv_metadata,...,UsZcta2010_FeatureClassCode,UsZcta2010_FunctionalStatus,UsZcta2010_GeoId,UsZcta2010_IsInside,UsZcta2010_LandAreaSqM,UsZcta2010_LatitudeDeg,UsZcta2010_LongitudeDeg,UsZcta2010_MinDistanceKm,UsZcta2010_WaterAreaSqM,UsZcta2010_ZctaCode
0,0,0,4413575,8 Mile Road,secondary,42.441909,-83.322163,42.441834,-83.321210,"{u'date': u'2016-08', u'status': u'OK', u'loca...",...,G6350,S,48152,true,31368242,42.424935,-83.374264,0,33600,48152
1,1,1,4688173,Eureka Road,secondary,42.192723,-83.348046,42.191222,-83.356192,"{u'date': u'2011-07', u'status': u'OK', u'loca...",...,G6350,S,48174,true,106680136,42.234298,-83.394324,0,588742,48174
2,2,4,4767192,Dickerson Street,tertiary,42.369270,-82.952822,42.370532,-82.953648,"{u'date': u'2013-09', u'status': u'OK', u'loca...",...,G6350,S,48215,true,10395926,42.375051,-82.954343,0,0,48215
3,3,5,4767193,Dickerson Street,tertiary,42.369270,-82.952822,42.366990,-82.950280,"{u'date': u'2013-09', u'status': u'OK', u'loca...",...,G6350,S,48215,true,10395926,42.375051,-82.954343,0,0,48215
4,4,7,4798440,Ecorse Road,tertiary,42.249202,-83.436907,42.248952,-83.430874,"{u'date': u'2016-08', u'status': u'OK', u'loca...",...,G6350,S,48184,true,18208616,42.274479,-83.395623,0,3148,48184


In [44]:
df3.to_csv('./streetview/data/google_street_view_metadata/wayne2-askgeo.csv', index=False)